In [65]:
import requests
import urllib.parse as ulp
import mysql.connector

In [71]:
mydb = mysql.connector.connect(
host="web5.ces.census.gov",
user=##,
password=##,
database="pmt"
)
mycursor = mydb.cursor()

sql = "SET collation_connection = 'utf8_general_ci'"
mycursor.execute(sql)
mydb.commit()
sql1 = "ALTER TABLE api_base CONVERT TO CHARACTER SET utf8 COLLATE utf8_general_ci"
mycursor.execute(sql1)
mydb.commit()

In [66]:
#https://api.base-search.net/cgi-bin/BaseHttpSearchInterface.fcgi?func=PerformSearch&query=%22Ronald%20Jarmin%22
import requests
import json
import urllib.parse as ulp
import urllib.request as ulr
import os
import subprocess as sp
import time

class Base:
        
    def sendToDB(self, proj_id, results, hits):
        #print(proj_id)
        mydb = mysql.connector.connect(
        host="web5.ces.census.gov",
        user="##",
        password="##",
        database="pmt"
        )

        mycursor = mydb.cursor()
        
        numPrint = 100
        if(hits < 100):
            numPrint = hits
        '''every quarter should add only new entries and update existing db entries
        if they changed-- use doc_id to ID which publications already in db'''
        #print("numPrint = " + str(numPrint))
        for i in range(numPrint):
            title = str(results[1]['response']['docs'][i].get('dctitle'))
            #print(title)
            authors = results[1]['response']['docs'][i].get('dcperson')
            if results[1]['response']['docs'][i].get('dcdoi') is not None:
                doi = results[1]['response']['docs'][i].get('dcdoi')[0]
            else: 
                doi = "None"
            if results[1]['response']['docs'][i].get('dcdate') is not None:
                pub_year = str(results[1]['response']['docs'][i].get('dcdate')[:4])
                pub_month = str(results[1]['response']['docs'][i].get('dcdate')[5:7])
            else:
                pub_year = "None"
                pub_month = "None"
            full_text_link = str(results[1]['response']['docs'][i].get('dcidentifier'))
            publisher = str(results[1]['response']['docs'][i].get('dcpublisher'))
            abstract = str(results[1]['response']['docs'][i].get('dcdescription'))
            doc_id = str(results[1]['response']['docs'][i].get('dcdocid'))
            #print(description)
            
            sql_dup = "SELECT * FROM api_base WHERE doc_id = %s AND proj_id = %s"
            data = (doc_id, proj_id)
            mycursor.execute(sql_dup, data)
            duplicates = mycursor.fetchall()
            #print(duplicates)
            
            
            #stop getting rid of duplicates!
            if len(duplicates) == 0:
                sql = "INSERT INTO api_base (proj_id, doi, title, pub_year, pub_month, publisher, abstract, full_text_link, doc_id) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)"
                val = (proj_id, doi, title, pub_year, pub_month, publisher, abstract, full_text_link, doc_id)
                mycursor.execute(sql, val)
                pub_id = mycursor.lastrowid
                print("pub_id: " +str(pub_id))

                mydb.commit()

                print(mycursor.rowcount, "record inserted.")

                if bool(authors):
                    for j in range(len(authors)):
                        sql_part = "INSERT INTO api_base_participants (pub_id, proj_id, fullname) VALUES (%s, %s, %s)"
                        val_part = (pub_id, proj_id, authors[j])
                        #sql_part = "INSERT INTO api_base_participants_test (proj_id, fullname) VALUES (%s, %s)"
                        #val_part = (proj_id, authors[j])
                        mycursor.execute(sql_part, val_part)

                        mydb.commit()

                        print(mycursor.rowcount, "participant record inserted.")
                    
            else: '''if duplicates in same query, 1st prob has most info. but if queries from separate quarters, prob want newer one-- how to logic?'''
                print("duplicate value, not added")
        
        
        sql_update_year = "UPDATE api_base SET pub_year=NULL WHERE pub_year='None'"
        mycursor.execute(sql_update_year)
        mydb.commit()
        sql_update_month = "UPDATE api_base SET pub_month=NULL WHERE pub_month='None'"
        mycursor.execute(sql_update_month)
        mydb.commit()
        sql_update_doi = "UPDATE api_base SET doi=NULL WHERE doi='None'"
        mycursor.execute(sql_update_doi)
        mydb.commit()
        sql_update_title = "UPDATE api_base SET title=NULL WHERE title='None'"
        mycursor.execute(sql_update_title)
        mydb.commit()
        sql_update_link = "UPDATE api_base SET full_text_link=NULL WHERE full_text_link='None'"
        mycursor.execute(sql_update_link)
        mydb.commit()
        sql_update_publisher = "UPDATE api_base SET publisher=NULL WHERE publisher='None'"
        mycursor.execute(sql_update_publisher)
        mydb.commit()
        sql_update_abstract = "UPDATE api_base SET abstract=NULL WHERE abstract='None'"
        mycursor.execute(sql_update_abstract)
        mydb.commit()
        sql_update_id = "UPDATE api_base SET doc_id=NULL WHERE doc_id='None'"
        mycursor.execute(sql_update_id)
        mydb.commit()
        print("changed None values to null in doi, title, year, month, full text link, publisher, abstract doc id")
        
class BaseSearch(Base):
            
    def search(self, _path, proj_id, _query):
        header = ["Accept: text/xml"]
        url = "https://api.base-search.net/cgi-bin/BaseHttpSearchInterface.fcgi"
        link = url+_path+"&query="+_query+"&hits=100&format=json"
        
        request = requests.get(link, proxies={'https':'##'})      
        
        if (request.status_code == 200):
            data = [proj_id, request.json()]
            return data
    
    '''change so do not need to input author name to run, check all aut names'''
    def searchAll(self, proj_id, _query):
        return self.search("?func=PerformSearch", proj_id, _query)


In [80]:
mydb = mysql.connector.connect(
        host="web5.ces.census.gov",
        user="##",
        password="##",
        database="pmt"
        )

mycursor = mydb.cursor()

test = "ALTER TABLE api_base AUTO_INCREMENT = 1"
mycursor.execute(test)
mydb.commit()
print("altered auto-increment")

altered auto-increment


In [ ]:
api = BaseSearch()
results = api.searchAll(1360, '"Ann Owens"')
hits = results[1]['response']['numFound']
print("Total hits: " + str(hits))
print("proj_id: " +str(results[0]))
proj_id = str(results[0])
print(results)
#api.sendToDB(proj_id, results, hits)

In [1]:
# !pip install tika
# from tika import parser # pip install tika
# raw = parser.from_file('abstract_5.pdf')
# print(raw['content'])

In [67]:
import nltk
nltk.set_proxy('http://proxy.tco##')
nltk.download("punkt")
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk import FreqDist

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\fang0311\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [68]:
from base64 import b64encode
from pprint import pprint
import requests

api_key = ##

headers = { 'Authorization' : 'key=%s' %  (api_key).decode("ascii") }

#https://www.ces.census.gov/api/iaa/v1.3/project/status/CP?fields=cms_project_id,title&page=0&limit=10
def run_query(path, fields, page, limit):
    request = requests.get('https://www.ces.census.gov/api/iaa/v1.3%s?fields=%s&page=%s&limit=%s' % (path,fields,page,limit), headers=headers, proxies={"https" : "148.129.75.18:3128"})
    if request.status_code == 200:
        return request.json()
    else:
        raise Exception("Query failed to run by returning code of {}.".format(request.status_code))

def name_and_ds(proj_id): 
    data = {}
    proj = run_query('/project/'+str(proj_id),'cms_project_id,title','0','10') # Execute the project query
    #'/project/status/CP'
    full = proj[0]['title']
    parts = full.split("-")
    data['title']= parts[2][1:]
    #print(proj['cms_project_id'])
    datasets = run_query('/project/%s/data' % proj[0]['cms_project_id'],'name','0','10') # Execute the dataset query
    sets = []

    for ds in datasets:
        sets.append(ds['name'])

    data['datasets']=sets
    return data


In [58]:
name_and_ds(5)

{'title': 'The Impact of Foreign Trade on the U.S. Economy',
 'datasets': ['Annual Survey of Manufactures',
  'Census of Manufactures',
  'Census of Mining',
  'Census of Construction Industries',
  'Enterprise Summary Report - ES9100 (large company)',
  'Auxiliary Establishment - ES9200',
  'Standard Statistical Establishment Listing ',
  'Census of Retail Trade',
  'Census of Services',
  'Census of Wholesale Trade']}

In [69]:
import base64
from base64 import b64encode
from pprint import pprint
import requests

api_key = ##

headers = { 'Authorization' : 'key=%s' %  (api_key).decode("ascii") }

def run__abstract_query(path,fields,page,limit):
    request = requests.get('https://www.ces.census.gov/api/iaa/v1.3%s?fields=%s&page=%s&limit=%s' % (path,fields,page,limit), headers=headers, proxies={"https" : "148.129.75.18:3128"})
    if request.status_code == 200:
        return request.json()
    else:
        raise Exception("Query failed to run by returning code of {}.".format(request.status_code))
        
def get_abstract(projid):
    results = run__abstract_query('/project/'+str(projid)+'/proposal/text','mime_data','0','10')
    #print(results)
    return results[0]['mime_data'].replace("\n", " ")

In [15]:
#get_abstract(5)

In [43]:
title = name_and_ds(712)['title']
description = title+' '+get_abstract(712)

#tokenizing words and getting rid of stop words
descrip = word_tokenize(description)

#nltk.download("stopwords")
#from nltk.corpus import stopwords
#from nltk.tokenize import word_tokenize
stop_words = set(stopwords.words("english"))

filtered_descrip = []
filtered_list = [
    word for word in descrip if word.casefold() not in stop_words]
#print(filtered_list)

#tagging POS
descrip_pos = nltk.pos_tag(descrip)

#chunking-- phrases with a specific format
#noun phrase that starts w/any # adjs, and ends w/any # nouns
grammar = "NP: {<JJ>*<NN>*}"
chunk_parser = nltk.RegexpParser(grammar)
tree = chunk_parser.parse(descrip_pos)
phrase = []
# stores all the phrases, key indexed by the # of words in the phrase
chunks = {}
for s in tree.subtrees(filter=lambda tree: tree.label() == 'NP'):
    #print([x[0] for x in s.leaves()])
    for x in s.leaves():
        phrase.append(x[0])
    #print(phrase)
    size = len(phrase)
    full = ""
    for i in range(size):
        if full=="":
            full = phrase[i]
        else:
            full = full+" "+phrase[i]
    if chunks.get(str(size)) is not None:
        chunks.get(str(size)).append(full)
    else:
        chunks[str(size)] = [full]
    phrase = []
#print("chunks")
#print(chunks)           
#tree.draw()

#freq distribution
freq_words = []
frequency_distribution = FreqDist(filtered_list)
#print(frequency_distribution.most_common(20))
for i in range(20):
    word = frequency_distribution.most_common(20)[i][0]
    if len(word) == 1:
        continue
    else:
        freq_words.append(word)
print("most frequent words")
print(freq_words)

#print(collections.counter())

#collocation- sequence of words that shows up often
# print("collocations")
# print(nltk.Text(description).collocation_list())

#ngrams, groups of n words
from nltk import ngrams, FreqDist
all_ngrams = {}
all_counts = dict()
for size in 2, 3, 4, 5:
    all_counts[size] = FreqDist(ngrams(filtered_list, size))
#print(all_counts[2].most_common(5))
third=''
fourth=''
fifth=''
for i in range(4):
    for j in range(5):
        first = all_counts[i+2].most_common(5)[j][0][0]
        second = all_counts[i+2].most_common(5)[j][0][1]
        if i+2 == 2:
            if len(first) == 1 or len(second) == 1:
                continue
            if all_ngrams.get(str(i+2)) is not None:
                all_ngrams.get(str(i+2)).append(first + " " + second)
            else:
                all_ngrams[str(i+2)] = [first + " " + second]
        if i+2 == 3:
            third = all_counts[i+2].most_common(5)[j][0][2]
            if len(first) == 1 or len(second) == 1 or len(third) == 1:
                continue
            if all_ngrams.get(str(i+2)) is not None:
                all_ngrams.get(str(i+2)).append(first + " " + second + " " + third)
            else:
                all_ngrams[str(i+2)] = [first + " " + second + " " + third]
        if i+2 == 4:
            third = all_counts[i+2].most_common(5)[j][0][2]
            fourth = all_counts[i+2].most_common(5)[j][0][3]
            if len(first) == 1 or len(second) == 1 or len(third) == 1 or len(fourth) == 1:
                continue
            if all_ngrams.get(str(i+2)) is not None:
                all_ngrams.get(str(i+2)).append(first + " " + second + " " + third + " " + fourth)
            else:
                all_ngrams[str(i+2)] = [first + " " + second + " " + third + " " + fourth]
        if i+2 == 5:
            third = all_counts[i+2].most_common(5)[j][0][2]
            fourth = all_counts[i+2].most_common(5)[j][0][3]
            fifth = all_counts[i+2].most_common(5)[j][0][4]
            if len(first) == 1 or len(second) == 1 or len(third) == 1 or len(fourth) == 1 or len(fifth) == 1:
                continue
            if all_ngrams.get(str(i+2)) is not None:
                all_ngrams.get(str(i+2)).append(first + " " + second + " " + third + " " + fourth + " " + fifth)
            else:
                all_ngrams[str(i+2)] = [first + " " + second + " " + third + " " + fourth + " " + fifth]
                
#print("ngrams")                
#print(all_ngrams)
#print(all_counts[3].most_common(5))

keywords = []
if all_ngrams.get("2") is not None:
    keywords += all_ngrams.get("2")
if chunks.get("2") is not None:
    keywords += chunks.get("2")
if chunks.get("3") is not None:
    keywords += chunks.get("3")
print("keywords w/duplicates")
#print(keywords)
print(len(keywords))

print("final keywords")
kw_set = set(keywords) #caps sensitive-- same words but dif capitalization don't get removed
keywords = list(kw_set)
#print(keywords)
print(len(keywords))

chunks
{'1': ['Uncertainty', 'exception', 'data', 'difficult', 'technology', 'cause', 'uncertainty', 'effect', 'uncertainty', 'response', 'necessary', 'analysis', 'employment', 'investment', 'research', 'development', 'productivity', 'project', 'project', 'quality', 'volatility', 'external', 'volatility', 'examination', 'assessment', 'aspect', 'quality', 'improvement', 'aggregate', 'data', 'analysis', 'significance', 'dispersion', 'project', 'recommendation', 'aggregate', 'yearly', 'proposal', 'interpretation', 'example', 'extent', 'volatility', 'recession', 'contribution', 'course', 'recession', 'non-response', 'volatility', 'volatility', 'cycle', 'project', 'cyclical', 'industrial', 'non-response', 'project', 'Research', 'time', 'volatility', 'example', 'industry', 'recession', 'population', 'amount', 'imputed', 'project', 'quality', 'many', 'Correspondence', 'literature', 'exception', 'difficult', 'large', 'ask', 'large', 'important', 'monetary', 'unanswered', 'technology', 'cause',

In [39]:
nlp_matches = {}

In [8]:
mydb = mysql.connector.connect(
        host="web5.ces.census.gov",
        user=##,
        password=##,
        database="pmt"
        )

mycursor = mydb.cursor()
    
from nltk import ngrams, FreqDist

#pjid = 712
nlp_data = []
nlp_rows = {}
nlp_res = {}

title = name_and_ds(pjid)['title']
description = title+' '+get_abstract(pjid)

'''NLP'''
#tokenizing words and getting rid of stop words
descrip = word_tokenize(description)

#nltk.download("stopwords")
#from nltk.corpus import stopwords
#from nltk.tokenize import word_tokenize
stop_words = set(stopwords.words("english"))

filtered_descrip = []
filtered_list = [
    word for word in descrip if word.casefold() not in stop_words]
#print(filtered_list)

#tagging POS
descrip_pos = nltk.pos_tag(descrip)

#chunking-- phrases with a specific format
#noun phrase that starts w/any # adjs, and ends w/any # nouns
grammar = "NP: {<JJ>*<NN>*}"
chunk_parser = nltk.RegexpParser(grammar)
tree = chunk_parser.parse(descrip_pos)
phrase = []
# stores all the phrases, key indexed by the # of words in the phrase
chunks = {}
for s in tree.subtrees(filter=lambda tree: tree.label() == 'NP'):
    #print([x[0] for x in s.leaves()])
    for x in s.leaves():
        phrase.append(x[0])
    #print(phrase)
    size = len(phrase)
    full = ""
    for i in range(size):
        if full=="":
            full = phrase[i]
        else:
            full = full+" "+phrase[i]
    if chunks.get(str(size)) is not None:
        chunks.get(str(size)).append(full)
    else:
        chunks[str(size)] = [full]
    phrase = []
#print("chunks")
#print(chunks)           
#tree.draw()

#freq distribution
freq_words = []
frequency_distribution = FreqDist(filtered_list)
#print(frequency_distribution.most_common(20))
for i in range(20):
    word = frequency_distribution.most_common(20)[i][0]
    if len(word) == 1:
        continue
    else:
        freq_words.append(word)
print("most frequent words")
print(freq_words)

#print(collections.counter())

#collocation- sequence of words that shows up often
# print("collocations")
# print(nltk.Text(description).collocation_list())

#ngrams, groups of n words
all_ngrams = {}
all_counts = dict()
for size in 2, 3, 4, 5:
    all_counts[size] = FreqDist(ngrams(filtered_list, size))
#print(all_counts[2].most_common(5))
third=''
fourth=''
fifth=''
for i in range(4):
    for j in range(5):
        first = all_counts[i+2].most_common(5)[j][0][0]
        second = all_counts[i+2].most_common(5)[j][0][1]
        if i+2 == 2:
            if len(first) == 1 or len(second) == 1:
                continue
            if all_ngrams.get(str(i+2)) is not None:
                all_ngrams.get(str(i+2)).append(first + " " + second)
            else:
                all_ngrams[str(i+2)] = [first + " " + second]
        if i+2 == 3:
            third = all_counts[i+2].most_common(5)[j][0][2]
            if len(first) == 1 or len(second) == 1 or len(third) == 1:
                continue
            if all_ngrams.get(str(i+2)) is not None:
                all_ngrams.get(str(i+2)).append(first + " " + second + " " + third)
            else:
                all_ngrams[str(i+2)] = [first + " " + second + " " + third]
        if i+2 == 4:
            third = all_counts[i+2].most_common(5)[j][0][2]
            fourth = all_counts[i+2].most_common(5)[j][0][3]
            if len(first) == 1 or len(second) == 1 or len(third) == 1 or len(fourth) == 1:
                continue
            if all_ngrams.get(str(i+2)) is not None:
                all_ngrams.get(str(i+2)).append(first + " " + second + " " + third + " " + fourth)
            else:
                all_ngrams[str(i+2)] = [first + " " + second + " " + third + " " + fourth]
        if i+2 == 5:
            third = all_counts[i+2].most_common(5)[j][0][2]
            fourth = all_counts[i+2].most_common(5)[j][0][3]
            fifth = all_counts[i+2].most_common(5)[j][0][4]
            if len(first) == 1 or len(second) == 1 or len(third) == 1 or len(fourth) == 1 or len(fifth) == 1:
                continue
            if all_ngrams.get(str(i+2)) is not None:
                all_ngrams.get(str(i+2)).append(first + " " + second + " " + third + " " + fourth + " " + fifth)
            else:
                all_ngrams[str(i+2)] = [first + " " + second + " " + third + " " + fourth + " " + fifth]

#print("ngrams")                
#print(all_ngrams)
#print(all_counts[3].most_common(5))

keywords = []
if all_ngrams.get("2") is not None:
    keywords += all_ngrams.get("2")
if chunks.get("2") is not None:
    keywords += chunks.get("2")
if chunks.get("3") is not None:
    keywords += chunks.get("3")
print("keywords w/duplicates")
#print(keywords)
print(len(keywords))

print("final keywords")
kw_set = set(keywords) #caps sensitive-- same words but dif capitalization don't get removed
keywords = list(kw_set)
#print(keywords)
print(len(keywords))

'''title keywords in description'''
q3 = "SELECT * FROM api_base WHERE proj_id= %s AND (description LIKE %s"
d3 = (pjid, keywords[0])
l3 = list(d3)
numData = len(keywords)
for i in range(numData-1):
    q3 += " OR description LIKE %s"
    if i == numData-2:
        q3 += ")"
    l3.append(str('%' + keywords[i+1] + '%'))
d3 = tuple(l3)
mycursor.execute(q3, d3)
r3 = mycursor.fetchall()
nlp_data.append(len(r3))
for i in range(len(r3)):
    if nlp_rows.get(r3[i][0]) is not None:
        nlp_rows[r3[i][0]] = nlp_rows.get(r3[i][0])+1
    else:
        nlp_rows[r3[i][0]] = 1


'''title keywords in title'''
q4 = "SELECT * FROM api_base WHERE proj_id= %s AND (title LIKE %s"
d4 = (pjid, keywords[0])
l4 = list(d4)
numData = len(keywords)
for i in range(numData-1):
    q4 += " OR title LIKE %s"
    if i == numData-2:
        q4 += ")"
    l4.append(str('%' + keywords[i+1] + '%'))
d4 = tuple(l4)
mycursor.execute(q4, d4)
r4 = mycursor.fetchall()
nlp_data.append(len(r4))
for i in range(len(r4)):
    if nlp_rows.get(r4[i][0]) is not None:
        nlp_rows[r4[i][0]] = nlp_rows.get(r4[i][0])+1
    else:
        nlp_rows[r4[i][0]] = 1

descrips = {}
for key in nlp_rows.keys():
    #print(type(key))
    q = "SELECT title FROM api_base WHERE pub_id = " +str(key)
    mycursor.execute(q)
    r = mycursor.fetchall()
    descrips[key] = [r[0][0]]
    qq = "SELECT description FROM api_base WHERE pub_id = " +str(key)
    mycursor.execute(qq)
    rr = mycursor.fetchall()
    descrips.get(key).append(rr[0][0])

#print(descrips)
print(nlp_rows)

for i in range(len(keywords)):
    for key in descrips.keys():
        if keywords[i] in descrips[key][0]:
            nlp_rows[key] = nlp_rows.get(key)+1
        if keywords[i] in descrips[key][1]:
            nlp_rows[key] = nlp_rows.get(key)+1

print(nlp_rows)
nlp_matches[pjid] = nlp_data
print(nlp_matches)
#nlp_res[pjid] = nlp_rows

In [9]:
search_NLP(5)

UnboundLocalError: local variable 'FreqDist' referenced before assignment

In [72]:
print(matches)

NameError: name 'matches' is not defined

In [61]:
test1 = ["foreign trade", "Foreign Trade"] 
test2 = set(test1)
print(list(test2))

['foreign trade', 'Foreign Trade']


In [45]:
#print(type(tree))

In [70]:
'''cms authors'''
import base64
from base64 import b64encode
from pprint import pprint
import requests

api_key = ##

headers = { 'Authorization' : 'key=%s' %  (api_key).decode("ascii") }

def run_query(path,fields,page,limit):
    request = requests.get('https://www.ces.census.gov/api/iaa/v1.3%s?fields=%s&page=%s&limit=%s' % (path,fields,page,limit), headers=headers, proxies={"https" : "148.129.75.18:3128"})
    if request.status_code == 200:
        return request.json()
    else:
        raise Exception("Query failed to run by returning code of {}.".format(request.status_code))

def get_authors(pjid):
    projects = run_query('/project/'+str(pjid),'cms_project_id,type_name,title,participants{username,fullname,roles}','0','100') # Execute the query

    auts = []
    
    for proj in projects:
        #print(proj['title'])
        if proj['title'].split("-")[0] == str(pjid) + " ":
            for part in proj['participants']:
                if 'researcher' in part['roles']:
                    auts.append(part['fullname'])
                    
    return auts

In [56]:
get_authors(5)

['J Bradford Jensen', 'Andrew B Bernard', 'Peter K Schott']

In [66]:
title = name_and_ds(712)['title']
description = title+' '+get_abstract(712)

#tokenizing words and getting rid of stop words
descrip = word_tokenize(description)

stop_words = set(stopwords.words("english"))

filtered_descrip = []
filtered_list = [
    word for word in descrip if word.casefold() not in stop_words]
#print(filtered_list)

#tagging POS
descrip_pos = nltk.pos_tag(descrip)

#chunking-- phrases with a specific format
#noun phrase that starts w/any # adjs, and ends w/any # nouns
grammar = "NP: {<JJ>*<NN>*}"
chunk_parser = nltk.RegexpParser(grammar)
tree = chunk_parser.parse(descrip_pos)
phrase = []
# stores all the phrases, key indexed by the # of words in the phrase
chunks = {}
for s in tree.subtrees(filter=lambda tree: tree.label() == 'NP'):
    #print([x[0] for x in s.leaves()])
    for x in s.leaves():
        phrase.append(x[0])
    #print(phrase)
    size = len(phrase)
    full = ""
    for i in range(size):
        if full=="":
            full = phrase[i]
        else:
            full = full+" "+phrase[i]
    if chunks.get(str(size)) is not None:
        chunks.get(str(size)).append(full)
    else:
        chunks[str(size)] = [full]
    phrase = []
# print("chunks")
# print(chunks)           
#tree.draw()

#freq distribution
freq_words = []
frequency_distribution = FreqDist(filtered_list)
#print(frequency_distribution.most_common(20))
for i in range(20):
    word = frequency_distribution.most_common(20)[i][0]
    if len(word) == 1:
        continue
    else:
        freq_words.append(word)
# print("most frequent words")
# print(freq_words)

#print(collections.counter())

#collocation- sequence of words that shows up often
# print("collocations")
# print(nltk.Text(description).collocation_list())

#ngrams, groups of n words
from nltk import ngrams, FreqDist
all_ngrams = {}
all_counts = dict()
for size in 2, 3, 4, 5:
    all_counts[size] = FreqDist(ngrams(filtered_list, size))
#print(all_counts[2].most_common(5))
third=''
fourth=''
fifth=''
for i in range(4):
    for j in range(5):
        first = all_counts[i+2].most_common(5)[j][0][0]
        second = all_counts[i+2].most_common(5)[j][0][1]
        if i+2 == 2:
            if len(first) == 1 or len(second) == 1:
                continue
            if all_ngrams.get(str(i+2)) is not None:
                all_ngrams.get(str(i+2)).append(first + " " + second)
            else:
                all_ngrams[str(i+2)] = [first + " " + second]
        if i+2 == 3:
            third = all_counts[i+2].most_common(5)[j][0][2]
            if len(first) == 1 or len(second) == 1 or len(third) == 1:
                continue
            if all_ngrams.get(str(i+2)) is not None:
                all_ngrams.get(str(i+2)).append(first + " " + second + " " + third)
            else:
                all_ngrams[str(i+2)] = [first + " " + second + " " + third]
        if i+2 == 4:
            third = all_counts[i+2].most_common(5)[j][0][2]
            fourth = all_counts[i+2].most_common(5)[j][0][3]
            if len(first) == 1 or len(second) == 1 or len(third) == 1 or len(fourth) == 1:
                continue
            if all_ngrams.get(str(i+2)) is not None:
                all_ngrams.get(str(i+2)).append(first + " " + second + " " + third + " " + fourth)
            else:
                all_ngrams[str(i+2)] = [first + " " + second + " " + third + " " + fourth]
        if i+2 == 5:
            third = all_counts[i+2].most_common(5)[j][0][2]
            fourth = all_counts[i+2].most_common(5)[j][0][3]
            fifth = all_counts[i+2].most_common(5)[j][0][4]
            if len(first) == 1 or len(second) == 1 or len(third) == 1 or len(fourth) == 1 or len(fifth) == 1:
                continue
            if all_ngrams.get(str(i+2)) is not None:
                all_ngrams.get(str(i+2)).append(first + " " + second + " " + third + " " + fourth + " " + fifth)
            else:
                all_ngrams[str(i+2)] = [first + " " + second + " " + third + " " + fourth + " " + fifth]
                
#print("ngrams")                
#print(all_ngrams)
#print(all_counts[3].most_common(5))

keywords = []
if all_ngrams.get("2") is not None:
    keywords += all_ngrams.get("2")
if chunks.get("2") is not None:
    keywords += chunks.get("2")
if chunks.get("3") is not None:
    keywords += chunks.get("3")
#print("keywords w/duplicates")
#print(keywords)
#print(len(keywords))

#print("final keywords")
kw_set = set(keywords) #caps sensitive-- same words but dif capitalization don't get removed
keywords = list(kw_set)
#print(keywords)
#print(len(keywords))
print("done")

done


In [86]:
def search_NLP(pjid, year):
    
    mydb = mysql.connector.connect(
        host="web5.ces.census.gov",
        user=##,
        password=##,
        database="pmt"
        )

    mycursor = mydb.cursor()
    
    import nltk
    #nltk.set_proxy('http://proxy.tco.census.gov:3128')
    #nltk.download("punkt")
    from nltk.corpus import stopwords
    from nltk.tokenize import sent_tokenize, word_tokenize
    from nltk import FreqDist
    from nltk import ngrams

    '''NLP'''
    title = name_and_ds(pjid)['title']
    description = title+' '+get_abstract(pjid)

    #tokenizing words and getting rid of stop words
    descrip = word_tokenize(description)

    stop_words = set(stopwords.words("english"))

    filtered_descrip = []
    filtered_list = [
        word for word in descrip if word.casefold() not in stop_words]
    #print(filtered_list)

    #tagging POS
    descrip_pos = nltk.pos_tag(descrip)

    #chunking-- phrases with a specific format
    #noun phrase that starts w/any # adjs, and ends w/any # nouns
    grammar = "NP: {<JJ>*<NN>*}"
    chunk_parser = nltk.RegexpParser(grammar)
    tree = chunk_parser.parse(descrip_pos)
    phrase = []
    # stores all the phrases, key indexed by the # of words in the phrase
    chunks = {}
    for s in tree.subtrees(filter=lambda tree: tree.label() == 'NP'):
        #print([x[0] for x in s.leaves()])
        for x in s.leaves():
            phrase.append(x[0])
        #print(phrase)
        size = len(phrase)
        full = ""
        for i in range(size):
            if full=="":
                full = phrase[i]
            else:
                full = full+" "+phrase[i]
        if chunks.get(str(size)) is not None:
            chunks.get(str(size)).append(full)
        else:
            chunks[str(size)] = [full]
        phrase = []
    # print("chunks")
    # print(chunks)           
    #tree.draw()

    #freq distribution
    freq_words = []
    frequency_distribution = FreqDist(filtered_list)
    #print(frequency_distribution.most_common(20))
    for i in range(20):
        word = frequency_distribution.most_common(20)[i][0]
        if len(word) == 1:
            continue
        else:
            freq_words.append(word)
    # print("most frequent words")
    # print(freq_words)

    #print(collections.counter())

    #collocation- sequence of words that shows up often
    # print("collocations")
    # print(nltk.Text(description).collocation_list())

    #ngrams, groups of n words
    from nltk import ngrams, FreqDist
    all_ngrams = {}
    all_counts = dict()
    for size in 2, 3, 4, 5:
        all_counts[size] = FreqDist(ngrams(filtered_list, size))
    #print(all_counts[2].most_common(5))
    third=''
    fourth=''
    fifth=''
    for i in range(4):
        for j in range(5):
            first = all_counts[i+2].most_common(5)[j][0][0]
            second = all_counts[i+2].most_common(5)[j][0][1]
            if i+2 == 2:
                if len(first) == 1 or len(second) == 1:
                    continue
                if all_ngrams.get(str(i+2)) is not None:
                    all_ngrams.get(str(i+2)).append(first + " " + second)
                else:
                    all_ngrams[str(i+2)] = [first + " " + second]
            if i+2 == 3:
                third = all_counts[i+2].most_common(5)[j][0][2]
                if len(first) == 1 or len(second) == 1 or len(third) == 1:
                    continue
                if all_ngrams.get(str(i+2)) is not None:
                    all_ngrams.get(str(i+2)).append(first + " " + second + " " + third)
                else:
                    all_ngrams[str(i+2)] = [first + " " + second + " " + third]
            if i+2 == 4:
                third = all_counts[i+2].most_common(5)[j][0][2]
                fourth = all_counts[i+2].most_common(5)[j][0][3]
                if len(first) == 1 or len(second) == 1 or len(third) == 1 or len(fourth) == 1:
                    continue
                if all_ngrams.get(str(i+2)) is not None:
                    all_ngrams.get(str(i+2)).append(first + " " + second + " " + third + " " + fourth)
                else:
                    all_ngrams[str(i+2)] = [first + " " + second + " " + third + " " + fourth]
            if i+2 == 5:
                third = all_counts[i+2].most_common(5)[j][0][2]
                fourth = all_counts[i+2].most_common(5)[j][0][3]
                fifth = all_counts[i+2].most_common(5)[j][0][4]
                if len(first) == 1 or len(second) == 1 or len(third) == 1 or len(fourth) == 1 or len(fifth) == 1:
                    continue
                if all_ngrams.get(str(i+2)) is not None:
                    all_ngrams.get(str(i+2)).append(first + " " + second + " " + third + " " + fourth + " " + fifth)
                else:
                    all_ngrams[str(i+2)] = [first + " " + second + " " + third + " " + fourth + " " + fifth]

    #print("ngrams")                
    #print(all_ngrams)
    #print(all_counts[3].most_common(5))

    keywords = []
    if all_ngrams.get("2") is not None:
        keywords += all_ngrams.get("2")
    if chunks.get("2") is not None:
        keywords += chunks.get("2")
    if chunks.get("3") is not None:
        keywords += chunks.get("3")
    #print("keywords w/duplicates")
    #print(keywords)
    #print(len(keywords))

    #print("final keywords")
    kw_set = set(keywords) #caps sensitive-- same words but dif capitalization don't get removed
    keywords = list(kw_set)
    #print(keywords)
    #print(len(keywords))
    print("done NLP")

    '''end NLP'''

    #description = "The project will proceed in three phases. The first will enhance the export and import information on Economic Censuses and Surveys. We will develop and test linkages between transaction level Foreign Trade data and Economic Census and Survey data and compare the links developed by CES to those developed by the Census Bureau’s Foreign Trade Division (FTD). If improvements in linkages methods are identified, we will provide advice to FTD. The second phase will analyze transaction level detail to examine changes in foreign trade transactions, focusing initially on related party transactions, potential underreporting, and impact of FTD outreach efforts. This component of the project will focus on identifying reasons for such changes. The third phase of the project will develop empirical and analytical framework to investigate the impact of trade on the U.S. economy - focusing on how firms allocate economic activity between domestic and foreign production and the impact of this on the domestic economy (including workers and regional economies). This component makes use of the data developed in the previous phases to develop new estimates of the impact of foreign trade on U.S. industries. It will examine how imports and exports affect domestic production, employment, and productivity. It will also examine how firm responses to trade affect local labor market outcomes."
    # key is author's name, values are the queries for that name
    # for each author name present, get an additional point

    '''inputs'''
    #pjid = 712 #given
    #year = 2009 #given

    res = {} # of points per pub_id for each pjid
    matches = {} # of hits for each query for each pjid
    authors = {}

    auts = get_authors(pjid) #array of authors' names
    numAuts = len(auts)
    title = name_and_ds(pjid)['title']
    # for each author, split into two search queries-- one w/ and one w/out middle initial

    for i in range(numAuts):
        part = auts[i].partition(' ')
        aut1 = auts[i]
        aut2 = part[0] + ' ' + part[2].partition(' ')[2]
        first = part[0]
        middle = part[2].partition(' ')[0]
        last = part[2].partition(' ')[2]
        #print(part[2].partition(' ')[2])
        if len(part[0]) == 1:
            first = part[0]+"."
        if len(part[2].partition(' ')[0]) == 1:
            middle = part[2].partition(' ')[0] + "."
        if len(part[2].partition(' ')[2]) == 1:
            last = part[2].partition(' ')[2] + "."
        aut3 = first + " " + last
        aut4 = first + " " + middle + " " + last
        authors[i] = [aut1, aut2, aut3, aut4]

    ds = name_and_ds(pjid)['datasets'] #array

    params_inner = {}
    params = {}

    params_inner['year'] = year
    params_inner['authors'] = auts
    params_inner['datasets'] = ds
    params_inner['keywords'] = keywords
    params[pjid] = params_inner

    #array of how many hits each query gets
    data = []

    '''pub_year'''
    q1 = "SELECT * FROM api_base WHERE proj_id= %s AND (pub_year >= %s OR pub_year is NULL)"
    d1 = (pjid, year)
    mycursor.execute(q1, d1)
    r1 = mycursor.fetchall()
    data.append(len(r1))
    rows = {}
    for i in range(len(r1)):
        if rows.get(r1[i][0]) is not None:
            rows[r1[i][0]] = rows.get(r1[i][0])+1
        else:
            rows[r1[i][0]] = 1
    #print(r1)
    #print(type(rows))
    #print(rows)

    '''authors'''
    aut_hits = 0

    def refineAut(pjid, aut1, aut2, aut3, aut4):
        q2 = "SELECT * FROM api_base_participants WHERE proj_id= %s AND (fullname = %s OR fullname = %s OR fullname = %s OR fullname = %s)"
        d2 = (pjid, aut1, aut2, aut3, aut4)
        mycursor.execute(q2, d2)
        r2 = mycursor.fetchall()
        #aut_hits += len(r2)
        for i in range(len(r2)):
            if rows.get(r2[i][0]) is not None:
                rows[r2[i][0]] = rows.get(r2[i][0])+1
        return len(r2)

    for i in range(numAuts):
        aut_hits += refineAut(pjid, authors.get(i)[0], authors.get(i)[1], authors.get(i)[2], authors.get(i)[3])
        #print(authors.get(i))
        #print(aut_hits)

    data.append(aut_hits)

    nlp_matches = {}
    nlp_data = []
    nlp_rows = {}
    nlp_res = {}

    '''queries 3 and 4: one pt each for having at least one matching word, 1 add. pt for each matching word'''
    '''title keywords in description'''
    q3 = "SELECT * FROM api_base WHERE proj_id= %s AND (description LIKE %s"
    d3 = (pjid, keywords[0])
    l3 = list(d3)
    numData = len(keywords)
    for i in range(numData-1):
        q3 += " OR description LIKE %s"
        if i == numData-2:
            q3 += ")"
        l3.append(str('%' + keywords[i+1] + '%'))
    d3 = tuple(l3)
    mycursor.execute(q3, d3)
    r3 = mycursor.fetchall()
    nlp_data.append(len(r3)) #amount of hits that have at least one matching word
    for i in range(len(r3)):
        if nlp_rows.get(r3[i][0]) is not None:
            nlp_rows[r3[i][0]] = nlp_rows.get(r3[i][0])+1
        else:
            nlp_rows[r3[i][0]] = 1

    '''title keywords in title'''
    q4 = "SELECT * FROM api_base WHERE proj_id= %s AND (title LIKE %s"
    d4 = (pjid, keywords[0])
    l4 = list(d4)
    numData = len(keywords)
    for i in range(numData-1):
        q4 += " OR title LIKE %s"
        if i == numData-2:
            q4 += ")"
        l4.append(str('%' + keywords[i+1] + '%'))
    d4 = tuple(l4)
    mycursor.execute(q4, d4)
    r4 = mycursor.fetchall()
    nlp_data.append(len(r4)) #amount of hits with at least one matching word
    for i in range(len(r4)):
        if nlp_rows.get(r4[i][0]) is not None:
            nlp_rows[r4[i][0]] = nlp_rows.get(r4[i][0])+1
        else:
            nlp_rows[r4[i][0]] = 1

    descrips = {}
    #grab and store titles and descriptions of all publications that had at least one match in title or descrip
    for key in nlp_rows.keys():
        #print(type(key))
        q = "SELECT title FROM api_base WHERE pub_id = " +str(key)
        mycursor.execute(q)
        r = mycursor.fetchall()
        descrips[key] = [r[0][0]]
        qq = "SELECT description FROM api_base WHERE pub_id = " +str(key)
        mycursor.execute(qq)
        rr = mycursor.fetchall()
        descrips.get(key).append(rr[0][0])

    #print(descrips)
    #print(nlp_rows)

    title_hits = 0
    descrip_hits = 0

    #print(keywords[0])
    #if descrips.keys() is not None:
    for i in range(len(keywords)):
        for key in descrips.keys():
            #print(key)
            #(keywords[i])
            #print(descrips[key][0])
            #print(key)
            #print(nlp_rows)
            if descrips[key][0] is not None:
                if keywords[i] in descrips[key][0]:
                    if nlp_rows[key] is not None:
                        nlp_rows[key] = nlp_rows.get(key)+1
                        title_hits += 1
            if descrips[key][1] is not None:
                if keywords[i] in descrips[key][1]:
                    if nlp_rows[key] is not None:
                        nlp_rows[key] = nlp_rows.get(key)+1
                        descrip_hits += 1

    data.append(title_hits)
    data.append(descrip_hits)

    #print(nlp_rows)
    nlp_matches[pjid] = nlp_data
    #print(nlp_matches)
    nlp_res[pjid] = nlp_rows


    '''datasets in description'''
    q5 = "SELECT * FROM api_base WHERE proj_id= %s AND (description LIKE %s"
    d5 = (pjid, ds[0])
    l5 = list(d5)
    numData = len(ds)
    for i in range(numData-1):
        q5 += " OR description LIKE %s"
        if i == numData-2:
            q5 += ")"
        l5.append(str('%' + ds[i+1] + '%'))
    d5 = tuple(l5)
    #print(q5)
    #print(d5)
    mycursor.execute(q5, d5)
    r5 = mycursor.fetchall()
    data.append(len(r5))
    for i in range(len(r5)):
        if rows.get(r5[i][0]) is not None:
            rows[r5[i][0]] = rows.get(r5[i][0])+1
        #else:
            #rows[r5[i][0]] = 1
    #print("r5:")
    #print(r5)
    #print(rows)

    for key in rows.keys():
        if nlp_rows.get(key) is not None:
            rows[key] = rows.get(key)+nlp_rows.get(key)
    #print(rows)

    res_sorted = {}

    for w in sorted(rows, key = rows.get, \
                    reverse = True):
        res_sorted[w] = rows[w]
        #print(w, rows[w])

    print("sorted results:")
    print(res_sorted)
    #print(rows)
    res[pjid] = rows
    matches[pjid] = data
    #print(res)
    print("matches:")
    print(matches)
    #print("params:")
    #print(params)
    #print("total hits: " + str(counter))

In [31]:
auts = get_authors(5)
part = auts[1].partition(' ')
first = part[0]
middle = part[2].partition(' ')[0]
last = part[2].partition(' ')[2]
#print(part[2].partition(' ')[2])
if len(part[0]) == 1:
    first = part[0]+"."
if len(part[2].partition(' ')[0]) == 1:
    middle = part[2].partition(' ')[0] + "."
if len(part[2].partition(' ')[2]) == 1:
    last = part[2].partition(' ')[2] + "."
print(first + " " + middle + " " + last)

Andrew B. Bernard


In [87]:
search_NLP(1284, 2015)

done NLP
sorted results:
{573: 9, 555: 7, 561: 7, 570: 7, 571: 7, 586: 7, 597: 7, 623: 7, 630: 7, 635: 7, 589: 6, 542: 5, 552: 5, 554: 5, 556: 5, 565: 5, 580: 5, 585: 5, 601: 5, 644: 5, 553: 4, 582: 4, 596: 4, 604: 4, 619: 4, 638: 4, 546: 3, 569: 3, 579: 3, 602: 3, 605: 3, 631: 3, 632: 3, 636: 3, 643: 3, 544: 2, 545: 2, 549: 2, 558: 2, 562: 2, 563: 2, 564: 2, 566: 2, 567: 2, 568: 2, 574: 2, 581: 2, 587: 2, 588: 2, 590: 2, 592: 2, 593: 2, 595: 2, 603: 2, 618: 2, 626: 2, 629: 2, 550: 1, 612: 1, 624: 1, 634: 1}
matches:
{1284: [61, 99, 1, 61, 0]}


In [58]:
print("res:")
print(res)
print("matches:")
print(matches)

res:
{5: {1: 2, 2: 2, 3: 2, 4: 1, 5: 2, 6: 2, 7: 2, 8: 2, 9: 2, 10: 2, 11: 2, 12: 2, 13: 2, 14: 2, 15: 2, 16: 2, 17: 1, 19: 1, 21: 2, 22: 2, 23: 2, 24: 2, 25: 2, 26: 2, 27: 2, 28: 2, 29: 2, 30: 2, 31: 2, 32: 2, 33: 2, 34: 2, 35: 2, 36: 2, 37: 2, 38: 2, 39: 2, 40: 2, 41: 2, 42: 2, 43: 2, 44: 2, 45: 2, 46: 2, 47: 2, 48: 2, 49: 2, 50: 2, 51: 2, 52: 2, 53: 1, 54: 2, 55: 2, 56: 2, 57: 2, 58: 2, 59: 2, 60: 2, 61: 2, 62: 2, 63: 2, 64: 2, 65: 2, 66: 2, 67: 2, 69: 2, 70: 2, 71: 2, 72: 2, 73: 2, 74: 2, 75: 2, 76: 2, 77: 2, 78: 2, 79: 2, 80: 2, 81: 2, 82: 2, 83: 2, 84: 2, 85: 1, 86: 2, 87: 2, 88: 3, 89: 2, 90: 2, 91: 2, 92: 2, 93: 1, 94: 2, 95: 2, 96: 2, 97: 2, 98: 2, 99: 2}, 343: {100: 2, 101: 2, 102: 2, 103: 2, 104: 1, 105: 2, 106: 2, 107: 2, 108: 2, 109: 2, 110: 3, 111: 2, 113: 1, 114: 1, 115: 1, 116: 2, 117: 2, 118: 2, 119: 1, 120: 2, 121: 3, 122: 2, 123: 2, 124: 2, 125: 2, 126: 2, 127: 2, 128: 1, 129: 1, 131: 1}, 580: {132: 2, 133: 2, 134: 2, 135: 2, 136: 2, 137: 2, 138: 2, 139: 2, 140: 2, 1

In [ ]:
# counter = 0
# for key in rows:
#     if rows.get(key) >= 2:
#         qe = "SELECT * FROM api_base WHERE proj_id= %s AND pub_id = %s"
#         de = (pjid, str(key))
#         mycursor.execute(qe, de)
#         print(mycursor.fetchall())
#         counter +=1
# print(">= 2: " + counter)
# for key in rows:
#     if rows.get(key) >= 3:
#         qe = "SELECT * FROM api_base WHERE proj_id= %s AND pub_id = %s"
#         de = (pjid, str(key))
#         mycursor.execute(qe, de)
#         print(mycursor.fetchall())
#         counter +=1
# print(">= 3: " + counter)
# for key in rows:
#     if rows.get(key) >= 4:
#         qe = "SELECT * FROM api_base WHERE proj_id= %s AND pub_id = %s"
#         de = (pjid, str(key))
#         mycursor.execute(qe, de)
#         print(mycursor.fetchall())
#         counter +=1
# print(">= 4: " + counter)
# for key in rows:
#     if rows.get(key) >= 5:
#         qe = "SELECT * FROM api_base WHERE proj_id= %s AND pub_id = %s"
#         de = (pjid, str(key))
#         mycursor.execute(qe, de)
#         print(mycursor.fetchall())
#         counter +=1
# print(">= 5: " + counter)
# for key in rows:
#     if rows.get(key) >= 6:
#         qe = "SELECT * FROM api_base WHERE proj_id= %s AND pub_id = %s"
#         de = (pjid, str(key))
#         mycursor.execute(qe, de)
#         print(mycursor.fetchall())
#         counter +=1
# print(">= 6: " + counter)

In [ ]:
'''inputs'''
pjid = 5
year = 2001
aut1 = "J. Bradford Jensen"
aut2 = "J. Jensen"
# two keywords from title to search in title and descriptions
kw1 = "%foreign trade%"
kw2 = "%U.S. economy%"
# four keywords/phrases from abstract to search in description
kw3 = "%party transactions%"
kw4 = "%domestic production%"
kw5 = "%local labor market%"
kw6 = "%FTD%"
ds1 = "%Annual Survey of Manufactures%"
ds = ['Annual Survey of Manufactures', 'Auxiliary Establishment - ES9200', 'Census of Construction Industries',
     'Census of Finance, Insurance, and Real Estate', 'Census of Manufactures', 'Census of Mining',
     'Census of Retail Trade', 'Census of Services', 'Census of Transportation, Communications, and Utilities',
     'Census of Wholesale Trade', 'Enterprise Summary Report - ES9100 (large company)', 'Standard Statistical Establishment Listing']

'''inputs'''
pjid = 343
year = 2004
aut1 = "Gale A. Boyd"
aut2 = "Gale Boyd"
# two keywords from title to search in title and descriptions
kw1 = "%modal choice%"
kw2 = "%product shipments%"
# four keywords/phrases from abstract to search in description
kw3 = "%forecasting capability%"
kw4 = "%shipment specific effects%"
kw5 = "%CFS%"
kw6 = "%NEMS%"
ds1 = "%Annual Survey of Manufactures%"
ds = ['Annual Survey of Manufactures', 'Commodity Flow Survey', 'Census of Manufactures',
     'Census of Mining', 'Census of Retail Trade', 'Census of Wholesale Trade',
     'Standard Statistical Establishment Listing']

'''inputs'''
pjid = 580
year = 2006
aut1 = "Jagadeesh M. Sivadasan"
aut2 = "Jagadeesh Sivadasan"
# two keywords from title to search in title and descriptions
kw1 = "%Micro-Productivity%"
kw2 = "%Heterogeneity%"
# four keywords/phrases from abstract to search in description
kw3 = "%IPO%"
kw4 = "%product market performance%"
kw5 = "%going public%"
kw6 = "%data quality%"
ds1 = "%Annual Capital Expenditures Survey%"
ds = ['Annual Capital Expenditures Survey', 'Annual Survey of Manufactures', 'Auxiliary Establishment - ES9200',
     'Business Expenditures Survey', 'Census of Manufactures', 'Compustat',
     'Census of Retail Trade', 'Longitudinal Business Database', 'Survey of Plant Capacity Utilization',
     'Survey of Industrial Research and Development', 'Survey of Manufacturing Technology', 'Standard Statistical Establishment Listing']

'''inputs'''
pjid = 861
year = 2011
aut1 = "Keren M. Horn"
aut2 = "Keren Horn"
# two keywords from title to search in title and descriptions
kw1 = "%neighborhood quality%"
kw2 = "%household residential choices%"
# four keywords/phrases from abstract to search in description
kw3 = "%neighborhood change%"
kw4 = "%racial composition%"
kw5 = "%zoned public school%"
kw6 = "%crime rate%"
ds1 = "%American Community Survey%"
ds = ['American Community Survey', 'American Housing Survey', 'Decennial Census']

'''inputs'''
pjid = 932
year = 2012
aut1 = "William R. Walker"
aut2 = "William Walker"
# two keywords from title to search in title and descriptions
kw1 = "%positive shocks%"
kw2 = "%prenatal conditions%"
# four keywords/phrases from abstract to search in description
kw3 = "%LEHD%"
kw4 = "%supplemental nutritional program%"
kw5 = "%clean air act%"
kw6 = "%fetal origins hypothesis%"
ds1 = "%Current Population Survey%"
ds = ['Current Population Survey', 'Current Population Survey (March/ASEC)', 'LEHD Employment History File',
     'LEHD Individual Characteristics File', 'Survey of Income and Program Participation']

'''inputs'''
pjid = 712
year = 2009
aut1 = "Nicholas A. Bloom"
aut2 = "Nicholas Bloom"
# two keywords from title to search in title and descriptions
kw1 = "%uncertainty%"
kw2 = "%business cycle%"
# four keywords/phrases from abstract to search in description
kw3 = "%time series%"
kw4 = "%volatility%"
kw5 = "%nonresponse%"
kw6 = "%cyclical%"
ds1 = "%Annual Survey of Manufactures%"
ds = ['Annual Survey of Manufactures', 'Census of Manufactures', 'Compustat', 
     'Longitudinal Business Database', 'Survey of Plant Capacity Utilization', 
     'Survey of Industrial Research and Development', 'Standard Statistical Establishment Listing']

'''inputs'''
pjid = 1195
year = 2014
aut1 = "Xavier A. Giroud"
aut2 = "Xavier Giroud"
# two keywords from title to search in title and descriptions
kw1 = "%resource allocation%"
kw2 = "%business cycle%"
# four keywords/phrases from abstract to search in description
kw3 = "%great recession%"
kw4 = "%capital stock imputation%"
kw5 = "%classication of establishments%"
kw6 = "%datasets%"
ds1 = "%Annual Survey of Manufactures%"
ds = ['Annual Survey of Manufactures', 'Auxiliary Establishment - ES9200', 'Census of Manufactures', 
     'Compustat', 'Longitudinal Business Database', 'LEHD Business Register Bridge', 
     'LEHD Employer Characteristics File', 'LEHD Employment History File', 'LEHD Geocoded Address List',
     'LEHD Individual Characteristics File', 'LEHD Unit-to-Worker', 'Standard Statistical Establishment Listing']

'''inputs'''
pjid = 1277
year = 2014
aut1 = "Richard Sander"
aut2 = "R. Sander"
# two keywords from title to search in title and descriptions
kw1 = "%labyrinth%"
kw2 = "%housing segregation%"
# four keywords/phrases from abstract to search in description
kw3 = "%residential segregation%"
kw4 = "%housing legislation%"
kw5 = "%black migration%"
kw6 = "%schelling process%"
ds = ['American Community Survey', 'Decennial Census']

'''inputs'''
pjid = 1284
year = 2015
aut1 = "Martha J. Bailey"
aut2 = "Martha Bailey"
# two keywords from title to search in title and descriptions
kw1 = "%long-run determinants%"
kw2 = "%processes%"
# four keywords/phrases from abstract to search in description
kw3 = "%ACS%"
kw4 = "%location of birth%"
kw5 = "%migration%"
kw6 = "%education%"
ds = ['American Community Survey', 'Decennial Census', 'BOC Census Numident', 'Disclosure Avoidance Population Tables']

'''inputs'''
pjid = 1360
year = 2015
aut1 = "Ann L. Owens"
aut2 = "Ann Owens"
# two keywords from title to search in title and descriptions
kw1 = "%trends%"
kw2 = "%hispanic neighborhood ascent%"
# four keywords/phrases from abstract to search in description
kw3 = "%socioeconomic ascent%"
kw4 = "%quantitative analyses%"
kw5 = "%metropolitan areas%"
kw6 = "%demographic processes%"
ds = ['American Community Survey', 'Decennial Census']

In [46]:
# api = Base()
# results = api.searchAll(0, '"Jack Johnson"')
# for i in range(5):
#     hits = results[1]['response']['numFound']
#     title = str(results[1]['response']['docs'][i]['dctitle'])
#     #doi = str(results[1]['data'][i]['_source']['doi'])
#     #work_id = str(results[1]['data'][i]['_id'])
#     authors = results[1]['response']['docs'][i]['dccreator']
#     #citations = str(results[1]['data'][i]['_source']['citations'])
#     pub_year = str(results[1]['response']['docs'][i]['dchdate'][:4])
#     pub_month = str(results[1]['response']['docs'][i]['dchdate'][5:7])
#     full_text_link = str(results[1]['response']['docs'][i]['dclink'])
#     description = str(results[1]['response']['docs'][i]['dcdescription'])
#     print("i: "+str(i))
#     print("hits: "+str(hits))
#     print("title: "+title)
#     print("authors: ")
#     for author in authors:
#         print(author)
#     print("pub_year: "+pub_year)
#     print("pub_month: "+pub_month)
#     print("full_text_link: "+full_text_link)
#     print("description: "+description)

In [2]:
# import urllib.request as urllib2

# proxy = urllib2.ProxyHandler({"https": "148.129.129.18:3128"})
# opener = urllib2.build_opener(proxy)
# urllib2.install_opener(opener)
# page = urllib2.urlopen("https://api.base-search.net/cgi-bin/BaseHttpSearchInterface.fcgi?func=PerformSearch&query=%22Ronald%20Jarmin%22&format=json")
# print(page.read())